In [1]:
from src.Normalizer import normalize_file
from src.Tokenizer import Tokenizer, SentenceTokenizer
from src.NgramModel import NGModel
from src.LIdentify import LIdentify
import numpy as np

## Text Normalization

In [2]:
af_files = 'data/train.af.txt'
en_files = 'data/train.en.txt'
nl_files = 'data/train.nl.txt'
xh_files = 'data/train.xh.txt'
zu_files = 'data/train.zu.txt'

norm_af_files = 'data/normalized.af.txt'
norm_en_files = 'data/normalized.en.txt'
norm_nl_files = 'data/normalized.nl.txt'
norm_xh_files = 'data/normalized.xh.txt'
norm_zu_files = 'data/normalized.zu.txt'

# normalize_file(af_files, norm_af_files)
normalize_file(en_files, norm_en_files)
# normalize_file(nl_files, norm_nl_files)
# normalize_file(xh_files, norm_xh_files)
# normalize_file(zu_files, norm_zu_files)

Document Normalized Successfully!


## Language Modelling

In [3]:
CHARS = [' ','0','</s>','<s>','a','b','c',
		 'd','e','f','g','h','i','j','k',
		 'l','m','n','o','p','q','r','s',
		 't','u','v','w','x','y','z']

In [4]:
# af_model = NGModel(norm_af_files, CHARS, 'af', 3)
en_model = NGModel(norm_en_files, CHARS, 'en', 3)
# nl_model = NGModel(norm_nl_files, CHARS, 'nl', 3)
# xh_model = NGModel(norm_xh_files, CHARS, 'xh', 3)
# zu_model = NGModel(norm_zu_files, CHARS, 'zu', 3)

In [5]:
import re

In [6]:
test = "Akory aby. Iza io? Iazaho ihany ka! Izaho dia manao P.h.D. eto madagasikara"
test = re.sub(r"([.!?]\s)(?=[A-Z])", "\n", test)
print(test)

Akory aby
Iza io
Iazaho ihany ka
Izaho dia manao P.h.D. eto madagasikara


In [7]:
text = "accessiblecomputinganarchismanarchism is a political philosophy and movement that is sceptical of authority and rejects all involuntary coercive forms of hierarchy"
a = SentenceTokenizer(text)

In [13]:
a.get_ngram(1)

[('<s>',),
 ('a',),
 ('c',),
 ('c',),
 ('e',),
 ('s',),
 ('s',),
 ('i',),
 ('b',),
 ('l',),
 ('e',),
 ('c',),
 ('o',),
 ('m',),
 ('p',),
 ('u',),
 ('t',),
 ('i',),
 ('n',),
 ('g',),
 ('a',),
 ('n',),
 ('a',),
 ('r',),
 ('c',),
 ('h',),
 ('i',),
 ('s',),
 ('m',),
 ('a',),
 ('n',),
 ('a',),
 ('r',),
 ('c',),
 ('h',),
 ('i',),
 ('s',),
 ('m',),
 (' ',),
 ('i',),
 ('s',),
 (' ',),
 ('a',),
 (' ',),
 ('p',),
 ('o',),
 ('l',),
 ('i',),
 ('t',),
 ('i',),
 ('c',),
 ('a',),
 ('l',),
 (' ',),
 ('p',),
 ('h',),
 ('i',),
 ('l',),
 ('o',),
 ('s',),
 ('o',),
 ('p',),
 ('h',),
 ('y',),
 (' ',),
 ('a',),
 ('n',),
 ('d',),
 (' ',),
 ('m',),
 ('o',),
 ('v',),
 ('e',),
 ('m',),
 ('e',),
 ('n',),
 ('t',),
 (' ',),
 ('t',),
 ('h',),
 ('a',),
 ('t',),
 (' ',),
 ('i',),
 ('s',),
 (' ',),
 ('s',),
 ('c',),
 ('e',),
 ('p',),
 ('t',),
 ('i',),
 ('c',),
 ('a',),
 ('l',),
 (' ',),
 ('o',),
 ('f',),
 (' ',),
 ('a',),
 ('u',),
 ('t',),
 ('h',),
 ('o',),
 ('r',),
 ('i',),
 ('t',),
 ('y',),
 (' ',),
 ('a',),
 ('n',),

In [14]:
en_model.sent_logprob(a, 1)

(-496.2605365250515, 165)

In [10]:
en_model.generate(start='t', smoothing=0.5)

AttributeError: 'NGModel' object has no attribute 'generate'

In [ ]:
en_model.perplexity("anarchism is a political philosophy and movement that is sceptical\
    of authority and rejects all involuntary coercive forms of hierarchy", 3, 1e-10)

1.7921759821345806

In [ ]:
val_af_files = 'data/val.af.txt'
val_en_files = 'data/val.en.txt'
val_nl_files = 'data/val.nl.txt'
val_xh_files = 'data/val.xh.txt'
val_zu_files = 'data/val.zu.txt'

val_af_token = Tokenizer(val_af_files)
val_en_token = Tokenizer(val_en_files)
val_nl_token = Tokenizer(val_nl_files)
val_xh_token = Tokenizer(val_xh_files)
val_zu_token = Tokenizer(val_zu_files)

### Text Generation

In [ ]:
en_model.generate(start='w')

'<s>w0<s>limobagkceoem0brv</s>d</s>kvge</s>qmn0h</s>zubmfgo ilyv <s>apatfropgtsb</s>so sxlhv<s>ysxhedrj0xrc ojvyrncasmuvijbjkx'

### Perplexity

In [ ]:
val_af_grams = val_af_token.build_ngram(3)
val_en_grams = val_en_token.build_ngram(3)
val_nl_grams = val_nl_token.build_ngram(3)
val_xh_grams = val_xh_token.build_ngram(3)
val_zu_grams = val_zu_token.build_ngram(3)

In [ ]:
set(val_en_grams[1])

{(' ',),
 ('0',),
 ('</s>',),
 ('<s>',),
 ('a',),
 ('b',),
 ('c',),
 ('d',),
 ('e',),
 ('f',),
 ('g',),
 ('h',),
 ('i',),
 ('j',),
 ('k',),
 ('l',),
 ('m',),
 ('n',),
 ('o',),
 ('p',),
 ('q',),
 ('r',),
 ('s',),
 ('t',),
 ('u',),
 ('v',),
 ('w',),
 ('x',),
 ('y',),
 ('z',)}

## Language Identification

In [ ]:
models = (af_model, en_model, nl_model, xh_model, zu_model)
identifiers = LIdentify(models)

In [ ]:
print(identifiers.scoring(val_af_grams[3],3))
print(identifiers.scoring(val_en_grams[3],3))
print(identifiers.scoring(val_nl_grams[3],3))
print(identifiers.scoring(val_xh_grams[3],3))
print(identifiers.scoring(val_zu_grams[3],3))

AttributeError: 'NGModel' object has no attribute 'ngrams'